In [1]:
# import necessary packages
import numpy as np
import torch
import torch.nn as nn

/Users/johnyang/anaconda3/envs/learn/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Section 3

### 11.5 FFNN in Programming

In [4]:
# define neural network
class Net(nn.Module):
    def __init__(self, input_size=2, hidden_dim1=2, hidden_dim2=3, hidden_dim3=2):
        super(Net, self).__init__()
        
        self.hidden1 = nn.Linear(input_size, hidden_dim1, bias=False)
        self.hidden1.weight = nn.Parameter(torch.tensor([[-2.,1.], [3.,-1.]]))
        
        self.hidden2 = nn.Linear(hidden_dim1, hidden_dim2, bias=False)
        self.hidden2.weight = nn.Parameter(torch.tensor([[0.,1.], [2.,-1.], [1.,2.]]))
        
        self.hidden3 = nn.Linear(hidden_dim2, hidden_dim3, bias=False)
        self.hidden3.weight = nn.Parameter(torch.tensor([[-1.,2.,1.], [3.,0.,0.]]))
        
        self.activation = nn.ReLU()
    
    # single step of forward propagation
    def forward(self, x):
        h1 = self.hidden1(x)
        h1 = self.activation(h1)
        h2 = self.hidden2(h1)
        h2 = self.activation(h2)
        h3 = self.hidden3(h2)
        return h3

In [5]:
net = Net()

# forward propagation with sample input
x = torch.tensor([3., 1.])
y_pred = net.forward(x)
print("Predicted value:", y_pred)

Predicted value: tensor([ 8., 24.], grad_fn=<SqueezeBackward3>)


In [6]:
# backpropagation with sample input
loss = nn.functional.cross_entropy(y_pred.unsqueeze(0), torch.LongTensor([1]))
loss.backward()
print(loss)
print(net.hidden1.weight.grad)

tensor(1.1921e-07, grad_fn=<NllLossBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00],
        [-7.3528e-07, -2.4509e-07]])


### 12.4 CNN in Programming

In [5]:
# import necessary packages
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [6]:
# set random seeds to ensure reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [8]:
# Normalize the dataset with a mean of 0.5 and standard deviation of 0.5 per color channel
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load CIFAR10 data
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data  = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 170498071/170498071 [00:25<00:00, 6790838.53it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [9]:
# object to iterate through train/test data in batches
train_loader = DataLoader(dataset=train_data, batch_size=8, shuffle=True,  num_workers=0)
test_loader  = DataLoader(dataset=test_data,  batch_size=8, shuffle=False, num_workers=0)

In [13]:
# define the CNN architecture
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # conv2d takes # input channels, # output channels, kernel size
        self.conv1 = nn.Conv2d(3, 3, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 16, 5)
        self.pool2 = nn.AvgPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [18]:
# - Run forward + back propagation on single data point -
# Note: Generally, we will train model on entire dataset multiple times

net = ConvNet()

# choose optimization technique
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# extract one image from dataset
images, labels = next(iter(train_loader))
image = images[0].unsqueeze(0) # Add one dimension to training data

# forward propagation
output = net(image)

# backpropagation
loss = torch.norm(output - torch.ones(output.shape[1]))**2 # squared error loss
loss.backward()       # calculate gradients of loss
optimizer.step()      # update params according to gradient descent
optimizer.zero_grad() # reset values of gradient to zero